## Základné nastavenia

Importujeme potrebné knižnice. Nasledujúci odstavec by mal byť spustený ako prvý predtým ako sa budú používať importované objekty `pd`, `np`, `plt` a `sns`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# nastavíme zobrazovanie grafov priamo v odstavcoch zápisníka
%matplotlib inline
# inicializujeme knižnicu seaborn
sns.set()

## Dátová množina - Titanic


[RMS Titanic](https://en.wikipedia.org/wiki/RMS_Titanic) bola britská osobná loď, ktorá stroskotala 14 apríla 1912 na svojej prvej komerčnej plavbe. V čase svojej služby bola najväčšou plávajúcou loďou na svete. Pri nehode zahynulo viac než 1500 osôb z celkového odhadovaného počtu 2224 pasažierov a členov posádky.

![RMS Titanic](titanic.jpg)
[Zdroj](https://en.wikipedia.org/wiki/Sinking_of_the_RMS_Titanic)

Dátová množina obsahuje informácie o pasažieroch Titanicu. Každý pasažier je popísaný nasledovnými atribútmi:
* `pclass` - trieda, v ktorej pasažier cestoval
* `survived` - udáva, či pasažier nehodu parníku prežil (1), alebo nie (0)
* `name` - meno cestujúceho
* `sex` - pohlavie
* `age` - vek
* `sibsp` - počet súrodencov pasažiera, resp. druhov/družiek
* `parch` - počet rodičov/detí medzi cestujúcimi
* `ticket` - číslo lístka
* `fare` - výška cestovného
* `cabin` - kajuta v ktorej bol cestujúci ubytovaný
* `embarked` - prístav, kde cestujúci nastúpil (`S` - Southampton, `C` - Cherbourg, `Q` - Queenstown)

In [ ]:
data = pd.read_csv("../data/titanic.csv")
print(data.shape)
print(data.columns)

In [ ]:
data.head()

### Úprava hodnôt

In [ ]:
data.drop(columns="ticket", inplace=True)

In [ ]:
data["embarked"] = data["embarked"].map({"S": "Southampton", "Q": 'Queenstown', "C":"Cherbourg"})
data["embarked"].value_counts()

### Nahradenie chýbajúcich hodnôt 1

In [ ]:
data.isna().sum()

In [ ]:
p = data["fare"].hist()

In [ ]:
fare_mean = data["fare"].mean()
fare_median = data["fare"].median()
print("fare mean: {0:.4f}, median: {1:.4f}".format(fare_mean, fare_median))

In [ ]:
data["fare"].fillna(fare_median, inplace=True)
data["fare"].isna().sum()

### Úloha 8.1

Chýbajúce hodnoty pre atribút `embarked` nahraďte najfrekventovanejšou hodnotou.

In [ ]:
# nahraďte chýbajúce hodnoty

# skontrolujeme, že všetky chýbajúce hodnoty atribútu embarked boli nahradené
data["embarked"].isna().sum()

### Odvodenie nových atribútov

In [ ]:
data["family_members"] = data.eval("sibsp + parch")
data["has_family"] = data.eval("family_members > 0")
data["has_family"].sum()

In [ ]:
def extract_title(name):
    if pd.isna(name):
        return np.nan
    start = name.find(",") + 2
    end = name.find(".")
    return name[start:end]

data["title"] = data["name"].apply(extract_title)
data.drop(columns="name", inplace=True)
data["title"].value_counts()

In [ ]:
def map_title(title):
    if title in {"Master", "Dr", "Rev", "Col", "Major", "Don", "Jonkheer", "Sir", "Dona", "Lady", "Capt", "the Countess"}:
        return "rare title"
    elif title in {"Mlle", "Ms"}:
        return "Miss"
    elif title in {"Mme"}:
        return "Mrs"
    return title;

data["title"] = data["title"].apply(map_title)
data["title"].value_counts()

### Nahradenie chýbajúcich hodnôt 2

In [ ]:
pd.pivot_table(data, index=["sex", "title"], values="age", aggfunc=["median", "mean", "min", "max", "count"], margins=True)

In [ ]:
l = data.query("sex == 'male' and title == 'rare title'")["age"].hist()

In [ ]:
def replace_missing_age(row):
    age = row["age"]
    sex = row["sex"]
    title = row["title"]
    if pd.isna(age):
        if sex == "female" and title == "Miss":
            return 24
        elif sex == "female" and title == "Mrs":
            return 32.5
        elif sex == "female" and title == "rare Title":
            return 43.5
        elif sex == "male" and title == "Mr":
            return 28
        elif sex == "male" and title == "rare Title":
            return 9.5
        else:
            return 28
    else:
        return age

age1 = data["age"].dropna()
age2 = data.apply(replace_missing_age, axis=1)
p = plt.hist([age1, age2])

In [ ]:
data["age"] = age2

### Prevedenie číselných atribútov na ordinálne - _diskretizácia hodnôt_

In [ ]:
data["fare"].describe()

In [ ]:
data["fare_ordinal"] = pd.cut(data["fare"], 3)
data["fare_ordinal"].value_counts()

In [ ]:
p = data["fare"].hist(bins=20)

In [ ]:
data["fare_ordinal"] = pd.qcut(data["fare"], 3)
data["fare_ordinal"].value_counts()

In [ ]:
data["fare_ordinal"] = pd.cut(data["fare"], bins=[0, 25, 100, 520])
data["fare_ordinal"].value_counts()

In [ ]:
data["fare_ordinal"] = pd.cut(data["fare"], bins=[0, 25, 100, 520], labels=["normal", "more expensive", "most expensive"])
data["fare_ordinal"].value_counts()

### Úloha 8.2

Odvoďte nový atribút `age_ordinal` diskretizovaním hodnôt `age` na intervaly 0-13, 13-19, 19-65, 65-maximálny vek s označeniami `child`, `young`, `adult`, `old`.

### Úloha 8.3

Označenia kajuty začínajú písmenom, ktoré označuje palubu na ktorej sa kajuta nachádzala (napr. kajuta `C22` sa nachádzala na palube `C`, atď.). Pomocou metódy `apply` odvoďte nový atribút `deck` s označením paluby.

Na ktorej palube by ste mali väčšiu šancu na záchranu?